# Exécuter des scripts en tant que travail de pipeline

Un pipeline vous permet de regrouper plusieurs étapes dans un même workflow. Vous pouvez créer un pipeline avec des composants. Chaque composant reflète un script Python à exécuter. Un composant est défini dans un fichier YAML qui spécifie le script et son mode d’exécution. 

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Créer les scripts

Vous allez créer un pipeline en deux étapes :

1. **Préparer les données** : Ajoutez les données manquantes et normalisez les données.
1. **Entraîner le modèle** : Entraînez un modèle de classification d’arbre de décision.

Exécutez les cellules suivantes pour créer le dossier **src** et les deux scripts.

In [ ]:
## Définir les composants

Pour définir le composant, vous devez spécifier :

- **Métadonnées** : *nom*, *com complet*, *version*, *description*, *type* etc. Les métadonnées permettent de décrire et de gérer le composant.
- **Interface** : *entrées* et *sorties*. Par exemple, un composant d’entraînement de modèle prend les données d’entraînement et le taux de régularisation comme entrée, et génère un fichier de modèle entraîné comme sortie. 
- **Commande, code et environnement** : *commande*, *code* et *environnement* pour exécuter le composant. La commande est la commande shell pour exécuter le composant. Le code fait généralement référence à un répertoire de code source. L’environnement peut être un environnement AzureML (organisé ou personnalisé), une image docker ou un environnement conda.

Exécutez les cellules suivantes pour créer un fichier YAML pour chaque composant que vous souhaitez exécuter en tant qu’étape de pipeline.

In [ ]:
## Charger les composants

Maintenant que vous avez défini chaque composant, vous pouvez les charger en référençant les fichiers YAML. 

## Générer le pipeline

Après avoir créé et chargé les composants, vous pouvez générer le pipeline. Vous allez définir les deux composants dans un pipeline. Tout d’abord, vous voulez que le composant `prep_data` soit exécuté. La sortie du premier composant doit être l’entrée du deuxième composant `train_decision_tree`, qui entraîne le modèle.

La fonction `diabetes_classification` représente le pipeline complet. La fonction attend une seule variable d'entrée : `pipeline_job_input`. Une ressource de données a été créée pendant l’installation. Vous allez utiliser la ressource de données inscrite comme entrée du pipeline. 

In [ ]:
Vous pouvez récupérer la configuration du travail de pipeline en imprimant l’objet `pipeline_job` :

In [ ]:
Vous pouvez changer n’importe quel paramètre de la configuration du travail de pipeline en vous reportant au paramètre et en spécifiant la nouvelle valeur :

In [ ]:
## Envoyer le travail du pipeline

Enfin, une fois que vous avez créé le pipeline et configuré le travail de pipeline pour qu’il s’exécute comme prévu, vous pouvez soumettre le travail de pipeline :

## Define the components

To define the component you need to specify:

- **Metadata**: *name*, *display name*, *version*, *description*, *type* etc. The metadata helps to describe and manage the component.
- **Interface**: *inputs* and *outputs*. For example, a model training component will take training data and the regularization rate as input, and generate a trained model file as output. 
- **Command, code & environment**: the *command*, *code* and *environment* to run the component. Command is the shell command to execute the component. Code usually refers to a source code directory. Environment could be an AzureML environment (curated or custom created), docker image or conda environment.

Run the following cells to create a YAML for each component you want to run as a pipeline step.

In [ ]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_file
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

In [ ]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train a decision tree classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_file
  reg_rate:
    type: number
    default: 0.01
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --reg_rate ${{inputs.reg_rate}} 
  --model_output ${{outputs.model_output}} 

## Load the components

Now that you have defined each component, you can load the components by referring to the YAML files. 

In [ ]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_decision_tree = load_component(source=parent_dir + "./train-model.yml")

## Build the pipeline

After creating and loading the components, you can build the pipeline. You'll compose the two components into a pipeline. First, you'll want the `prep_data` component to run. The output of the first component should be the input of the second component `train_decision_tree`, which will train the model.

The `diabetes_classification` function represents the complete pipeline. The function expects one input variable: `pipeline_job_input`. A data asset was created during setup. You'll use the registered data asset as the pipeline input. 

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def diabetes_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_decision_tree(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = diabetes_classification(Input(type=AssetTypes.URI_FILE, path="azureml:diabetes-data:1"))

You can retrieve the configuration of the pipeline job by printing the `pipeline_job` object:

In [ ]:
print(pipeline_job)

You can change any parameter of the pipeline job configuration by referring to the parameter and specifying the new value:

In [ ]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "aml-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

## Submit the pipeline job

Finally, when you've built the pipeline and configured the pipeline job to run as required, you can submit the pipeline job:

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_diabetes"
)
pipeline_job